In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import sys

In [3]:
sys.path.append("/home/lemma/Documents/MAC-POSTS/side_project/network_builder")

In [4]:
from models import *
import MNMAPI
from MNM_mcnb import *

In [5]:
target_time = 3600
early_penalty = 0.7
late_penalty = 10

In [6]:
# s = driving_route(target_time, early_penalty, late_penalty, [0,0,0], [1, 2, 4], 1, 3, 120)

In [7]:
# single_driving_config = dict()
# single_driving_config['mode'] = 'single_drive'
# single_driving_config['target_time'] = target_time
# single_driving_config['early_penalty'] = early_penalty
# single_driving_config['late_penalty'] = late_penalty
# single_driving_config['ID_list'] = [0, 0, 0]
# single_driving_config['O'] = 0
# single_driving_config['D'] = 1
# single_driving_config['link_list'] = [0, 1, 2]
# single_driving_config['path_ID'] = 0
# single_driving_config['number_people'] = 1
# single_driving_config['parking_lot'] = parking_lot(5)
# single_driving_config['walking_time1'] = 3
# single_driving_config['walking_time2'] = 3
# p1 = make_path(single_driving_config)

In [8]:
# multiple_driving_config = dict()
# multiple_driving_config['mode'] = 'multiple_drive'
# multiple_driving_config['target_time'] = target_time
# multiple_driving_config['early_penalty'] = early_penalty
# multiple_driving_config['late_penalty'] = late_penalty
# multiple_driving_config['ID_list'] = [0, 0, 0]
# multiple_driving_config['O'] = 0
# multiple_driving_config['D'] = 1
# multiple_driving_config['link_list'] = [1, 2, 3]
# multiple_driving_config['path_ID'] = 1
# multiple_driving_config['number_people'] = 2
# multiple_driving_config['parking_lot'] = parking_lot(5)
# multiple_driving_config['walking_time1'] = 3
# multiple_driving_config['walking_time2'] = 3
# p2 = make_path(multiple_driving_config)

In [9]:
# transit_config = dict()
# transit_config['mode'] = 'transit'
# transit_config['target_time'] = target_time
# transit_config['early_penalty'] = early_penalty
# transit_config['late_penalty'] = late_penalty
# transit_config['ID_list'] = [0, 0, 0]
# transit_config['O'] = 0
# transit_config['D'] = 1
# transit_config['link_list'] = [2, 4, 5]
# transit_config['path_ID'] = 2
# transit_config['transit_fare'] = 5
# transit_config['walking_time1'] = 4
# transit_config['walking_time2'] = 5
# transit_config['transit_time'] = 40
# p3 = make_path(transit_config)

In [10]:
# metro_config = dict()
# metro_config['mode'] = 'metro'
# metro_config['target_time'] = target_time
# metro_config['early_penalty'] = early_penalty 
# metro_config['late_penalty'] = late_penalty
# metro_config['O'] = 0
# metro_config['D'] = 1
# metro_config['ID_list'] = [0,1,3]
# metro_config['walking_time1'] = 40
# metro_config['metro_time'] = 100
# metro_config['walking_time2'] = 30
# metro_config['metro_fee'] = 5
# p4 = make_path(metro_config)

In [11]:
# pnr_config = dict()
# pnr_config['mode'] = 'pnr'
# pnr_config['target_time'] = target_time
# pnr_config['early_penalty'] = early_penalty
# pnr_config['late_penalty'] = late_penalty
# pnr_config['ID_list'] = [0, 0, 0]
# pnr_config['O'] = 0
# pnr_config['D'] = 1
# pnr_config['car_link_list'] = [1, 2, 3]
# pnr_config['car_path_ID'] = 1
# pnr_config['transit_link_list'] = [1, 2, 3]
# pnr_config['transit_path_ID'] = 4
# pnr_config['parking_lot'] = parking_lot(3)
# pnr_config['transit_fare'] = 5
# pnr_config['before_drive_walking_time'] = 3
# pnr_config['switching_time'] = 5
# pnr_config['after_transit_walking_time'] = 1
# pnr_config['transit_time'] = 2
# p5 = make_path(pnr_config)

In [12]:
# a = MNMAPI.mcdta_api()

## Test NB

In [13]:
from exp_config import *

Creating route for: single_drive
Creating route for: single_drive
Creating route for: multiple_drive
Creating route for: metro
Creating route for: pnr
Creating route for: pnr
Creating route for: pnr
Creating route for: pnr
Creating route for: single_drive
Creating route for: single_drive
Creating route for: multiple_drive
Creating route for: transit
Creating route for: pnr
Creating route for: single_drive
Creating route for: single_drive
Creating route for: multiple_drive
Creating route for: pnr


In [14]:
data_folder = '../data/input_files_small_multiclass'

In [15]:
nb = MNM_network_builder()

In [16]:
nb.load_from_folder(data_folder)

In [17]:
num_assign_intervals = nb.config.config_dict['DTA']['max_interval']

In [18]:
# demand_dict = dict()
# demand_dict[1] = dict()
# demand_dict[2] = dict()
# demand_dict[3] = dict()
# demand_dict[4] = dict()
# demand_dict[5] = dict()
# demand_dict[1][1] = [np.ones(num_assign_intervals) * 10, np.zeros(num_assign_intervals)]
# demand_dict[2][1] = [np.zeros(num_assign_intervals), np.ones(num_assign_intervals) * 1]
# demand_dict[3][1] = [np.ones(num_assign_intervals) * 10, np.zeros(num_assign_intervals)]
# demand_dict[4][1] = [np.ones(num_assign_intervals) * 10, np.zeros(num_assign_intervals)]
# demand_dict[5][1] = [np.zeros(num_assign_intervals), np.ones(num_assign_intervals) * 10]
# demand_dict[1][2] = [np.ones(num_assign_intervals) * 10, np.zeros(num_assign_intervals)]
# demand_dict[1][3] = [np.ones(num_assign_intervals) * 10, np.zeros(num_assign_intervals)]
# demand_dict[3][2] = [np.ones(num_assign_intervals) * 10, np.zeros(num_assign_intervals)]
# demand_dict[4][3] = [np.ones(num_assign_intervals) * 10, np.zeros(num_assign_intervals)]

In [19]:
# nb.path_table.ID2path

In [20]:
# path_matrix = np.random.rand(len(path_list), num_assign_intervals) * 10

In [21]:
# def get_uniform_mcflow(demand_dict, path_dict, ID2path, MNM_od, num_assign_intervals):
#     f_car = np.ones((len(ID2path), num_assign_intervals))
#     f_truck = np.ones((len(ID2path), num_assign_intervals))
#     for O_node in path_dict.keys():
#         O = MNM_od.O_dict.inv[O_node]
#         for D_node in path_dict[O_node].keys():
#             D = MNM_od.D_dict.inv[D_node]
#             path_list = path_dict[O_node][D_node].path_list
#             num_path = len(path_list)
#             for path in path_list:
#                 idx = ID2path.keys().index(path.path_ID)
#                 f_car[idx, :] = demand_dict[O][D][0] / np.float(num_path)
#                 f_truck[idx, :] = demand_dict[O][D][1] / np.float(num_path)
#     return f_car.flatten(order = "F"), f_truck.flatten(order = "F")

In [22]:
# f_car, f_truck = get_uniform_mcflow(demand_dict, nb.path_table.path_dict, nb.path_table.ID2path, nb.od, num_assign_intervals)

In [23]:
# mcdue.num_assign_interval = num_assign_intervals
# mcdue.simulation_path_list = range(nb.config.config_dict['FIXED']['num_path'])
# mcdue.num_simulation_path = nb.config.config_dict['FIXED']['num_path']

In [24]:
# car_flow, truck_flow = mcdue.form_demand_for_simulation(path_list, path_matrix)

In [25]:
# nb.update_demand_path(car_flow.flatten(order = 'F'), truck_flow.flatten(order = 'F'), choice_dict)

In [26]:
# nb.dump_to_folder("test")

In [27]:
# link_ID_list = list(map(lambda x: x.ID, nb.link_list))

In [28]:
# dta = MNMAPI.mcdta_api()
# dta.initialize('test')
# dta.register_links(link_ID_list)
# dta.run_whole()

In [29]:
nb.config.config_dict['DTA']['assign_frq']

180

In [30]:
mcdue = Multimode_DUE(nb)

Init simulation


In [31]:
init_path_matrix = mcdue.init_path_matrix(path_list, demand_dict)

In [ ]:
path_matrix = mcdue.solve(init_path_matrix, path_list, demand_dict, ab_dict, choice_dict)

1
2
11
12
13
14
15
16
17
18


In [ ]:
# car_flow, truck_flow = mcdue.form_demand_for_simulation(path_list, path_matrix)

In [ ]:
# dta = mcdue.get_simulation(car_flow, truck_flow, choice_dict)

In [ ]:
# dta.get_cur_loading_interval()

In [ ]:
# c = mcdue.get_cost_matrix(dta, path_list)

In [36]:
# Lambda_matrix = mcdue.get_Lambda_matrix(dta, path_list, path_matrix, demand_dict, ab_dict)

In [37]:
# mcdue.get_merit_gap(Lambda_matrix, path_matrix, path_list, demand_dict)

2157633.513948284

In [39]:
# new_path_matrix = mcdue.update_path_matrix(Lambda_matrix, path_matrix, path_list, demand_dict)

In [42]:
# new_Lambda_matrix = mcdue.get_Lambda_matrix(dta, path_list, new_path_matrix, demand_dict, ab_dict)
# mcdue.get_merit_gap(new_Lambda_matrix, new_path_matrix, path_list, demand_dict)

2112681.11862898

## Due

In [76]:
num_interval = 100
path_list = [p1, p2, p3, p4, p5]
num_paths = len(path_list)

In [77]:
mcdue = Multimode_DUE()

Init simulation


In [78]:
mcdue.num_simulation_path = 4
mcdue.num_assign_interval = num_interval
mcdue.simulation_path_list = [0, 1, 2, 3]

In [79]:
mcdue.form_demand_for_simulation(path_list, np.random.rand(num_paths, num_interval))

driving route
driving route
park and ride route


(array([[0.8931231 , 0.50475542, 0.28551118, 0.66126754, 0.09285747,
         0.26125859, 0.15250134, 0.47519616, 0.2730772 , 0.43813649,
         0.92410653, 0.10276965, 0.17779759, 0.59778608, 0.80281787,
         0.99845093, 0.0793874 , 0.35896397, 0.64572945, 0.98442713,
         0.76938174, 0.47968491, 0.92509428, 0.3406183 , 0.68278708,
         0.28881665, 0.13676748, 0.70977973, 0.31356856, 0.22354162,
         0.2241553 , 0.56145298, 0.14467505, 0.70310654, 0.09865757,
         0.70259457, 0.1018712 , 0.90556787, 0.33208169, 0.25035887,
         0.00190799, 0.49232597, 0.74318139, 0.93522484, 0.96831422,
         0.45078214, 0.47162309, 0.98954834, 0.89823852, 0.7719648 ,
         0.31668399, 0.42485018, 0.38659821, 0.41642567, 0.39397644,
         0.50124378, 0.53906867, 0.09935292, 0.35170263, 0.74636223,
         0.42040339, 0.19450825, 0.19355516, 0.69194091, 0.69293419,
         0.86196369, 0.06564493, 0.88074224, 0.76213871, 0.81406661,
         0.27081251, 0.72376401, 0

In [86]:
path_list